In [1]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm_notebook
import tqdm
import seaborn as sns
from choices import param_dict
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import  IterativeImputer 
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier
sns.set(font_scale=1.3, rc={'axes.facecolor':'white', 'figure.facecolor':'white' , 'axes.grid':True} , style="whitegrid")
# 
import time 
tstart = time.time()
feat_to_drop = param_dict['hardness']+param_dict['IRAC']

In [2]:

from utilities import deets
from choices import get_train_data , param_dict
classes = ['AGN' ,'STAR' , 'YSO' ,  'CV' , 'LMXB' , 'HMXB' ,'ULX','PULSAR']
flag = {
    'conf_flag' : 0 , 
    'streak_src_flag' : 0 , 
    'extent_flag' : 0 , 
    'pileup_flag' : 0 , 
    }
ret_dict= {
    'clf': True,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : True
 }



In [6]:

file = f'../compiled_data_v3/imputed_data_v2/x_phot_minmax_modeimp.csv'
data = get_train_data(flags = flag, classes= classes , offset = 1, file=file)
data = data.drop(columns = feat_to_drop)
deets(data,0)

from utilities import cv
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1)
res_final  = cv(
    {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
    {'model' : rf , 'name' :'GB'} , ## CHANGE HERE########################################################
    k=20 , return_dict = ret_dict, save_df= f'' , multiprocessing = False)

_____________________________________________________
------------------------------
Number of Objects : 7703
Number of Columns : 42
_____________________________________________________
Doing 20 fold cross-validation


100%|██████████| 20/20 [02:23<00:00,  7.15s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
>>> Training the final classifier RandomForestClassifier(max_depth=30, n_estimators=400, n_jobs=-1)
[DONE] Classifier is trained | acces via ['clf'] in the response


In [9]:
res_final['pr_score']

,class,precision,recall
0,HMXB,0.854278,0.804786
1,AGN,0.967432,0.942636
2,YSO,0.937337,0.898998
3,STAR,0.953405,0.938603
4,LMXB,0.741259,1.000000
5,ULX,0.540284,0.712500
6,CV,0.415663,0.644860
7,PULSAR,0.198020,0.434783
